This is a low cholesterol diet model: objective is to minimize cholesterol intake.

In [1]:
# import required packages
import pandas as pd
from pulp import *
import numpy as np

In [2]:
# import and prepare data
dietlarge0 = pd.read_excel('diet_large.xls', sheet_name = 'Sheet1')
dietlarge0.columns = ['Long_Desc', 'Protein', 'Carbohydrate, by difference', 'Energy1',
       'Water', 'Energy2', 'Calcium, Ca', 'Iron, Fe', 'Magnesium, Mg',
       'Phosphorus, P', 'Potassium, K', 'Sodium, Na', 'Zinc, Zn', 'Copper, Cu',
       'Manganese, Mn', 'Selenium, Se', 'Vitamin A, RAE',
       'Vitamin E (alpha-tocopherol)', 'Vitamin D',
       'Vitamin C, total ascorbic acid', 'Thiamin', 'Riboflavin', 'Niacin',
       'Pantothenic acid', 'Vitamin B-6', 'Folate, total', 'Vitamin B-12',
       'Vitamin K (phylloquinone)', 'Cholesterol', 'Fatty acids, total trans',
       'Fatty acids, total saturated']
dietlarge0.head(5)

,Long_Desc,Protein,"Carbohydrate, by difference",Energy1,Water,Energy2,"Calcium, Ca","Iron, Fe","Magnesium, Mg","Phosphorus, P",...,Riboflavin,Niacin,Pantothenic acid,Vitamin B-6,"Folate, total",Vitamin B-12,Vitamin K (phylloquinone),Cholesterol,"Fatty acids, total trans","Fatty acids, total saturated"
0,Long_Desc,Protein,"Carbohydrate, by difference",Energy,Water,Energy,"Calcium, Ca","Iron, Fe","Magnesium, Mg","Phosphorus, P",...,Riboflavin,Niacin,Pantothenic acid,Vitamin B-6,"Folate, total",Vitamin B-12,Vitamin K (phylloquinone),Cholesterol,"Fatty acids, total trans","Fatty acids, total saturated"
1,"Butter, salted",0.85,0.06,717,15.87,3000,24,0.02,2,24,...,0.034,0.042,0.11,0.003,3,0.17,7,215,NaN,51.368
2,"Butter, whipped, with salt",0.85,0.06,717,15.87,2999,24,0.16,2,23,...,0.034,0.042,0.11,0.003,3,0.13,7,219,NaN,50.489
3,"Butter oil, anhydrous",0.28,0,876,0.24,3665,4,0,0,3,...,0.005,0.003,0.01,0.001,0,0.01,8.6,256,NaN,61.924
4,"Cheese, blue",21.4,2.34,353,42.41,1477,528,0.31,23,387,...,0.382,1.016,1.729,0.166,36,1.22,2.4,75,NaN,18.669


In [3]:
dietlarge = dietlarge0[1:7147]
dietlarge = dietlarge.fillna(0).reset_index(drop = True)
dietlarge.head()

,Long_Desc,Protein,"Carbohydrate, by difference",Energy1,Water,Energy2,"Calcium, Ca","Iron, Fe","Magnesium, Mg","Phosphorus, P",...,Riboflavin,Niacin,Pantothenic acid,Vitamin B-6,"Folate, total",Vitamin B-12,Vitamin K (phylloquinone),Cholesterol,"Fatty acids, total trans","Fatty acids, total saturated"
0,"Butter, salted",0.85,0.06,717,15.87,3000,24,0.02,2,24,...,0.034,0.042,0.110,0.003,3,0.17,7.0,215,0.0,51.368
1,"Butter, whipped, with salt",0.85,0.06,717,15.87,2999,24,0.16,2,23,...,0.034,0.042,0.110,0.003,3,0.13,7.0,219,0.0,50.489
2,"Butter oil, anhydrous",0.28,0.00,876,0.24,3665,4,0.00,0,3,...,0.005,0.003,0.010,0.001,0,0.01,8.6,256,0.0,61.924
3,"Cheese, blue",21.40,2.34,353,42.41,1477,528,0.31,23,387,...,0.382,1.016,1.729,0.166,36,1.22,2.4,75,0.0,18.669
4,"Cheese, brick",23.24,2.79,371,41.11,1552,674,0.43,24,451,...,0.351,0.118,0.288,0.065,20,1.26,2.5,94,0.0,18.764


In [4]:
dailyintakelarge = dietlarge0[-3:].reset_index(drop = True)
dailyintakelarge

,Long_Desc,Protein,"Carbohydrate, by difference",Energy1,Water,Energy2,"Calcium, Ca","Iron, Fe","Magnesium, Mg","Phosphorus, P",...,Riboflavin,Niacin,Pantothenic acid,Vitamin B-6,"Folate, total",Vitamin B-12,Vitamin K (phylloquinone),Cholesterol,"Fatty acids, total trans","Fatty acids, total saturated"
0,NaN,56,130,2400,3700,2400,1000,8,270,700,...,1.3,16,5,1.3,400,2.4,120,NaN,NaN,NaN
1,NaN,g/d,g/d,kcal,g,NaN,mg/d,mg/d,mg/d,mg/d,...,mg/d,mg/d,mg/d,mg/d,microg/d,microg/d,microg/d,NaN,NaN,NaN
2,NaN,1000000,1000000,1000000,1000000,1000000,2500,45,400,4000,...,1000000,35,1000000,100,1000,1000000,1000000,NaN,NaN,NaN


In [5]:
foodlarge = dietlarge.Long_Desc.tolist()
foodlarge[:5]

['Butter, salted',
 'Butter, whipped, with salt',
 'Butter oil, anhydrous',
 'Cheese, blue',
 'Cheese, brick']

In [6]:
all_data_large = {}
for j in ['Protein', 'Carbohydrate, by difference', 'Energy1',
       'Water', 'Energy2', 'Calcium, Ca', 'Iron, Fe', 'Magnesium, Mg',
       'Phosphorus, P', 'Potassium, K', 'Sodium, Na', 'Zinc, Zn', 'Copper, Cu',
       'Manganese, Mn', 'Selenium, Se', 'Vitamin A, RAE',
       'Vitamin E (alpha-tocopherol)', 'Vitamin D',
       'Vitamin C, total ascorbic acid', 'Thiamin', 'Riboflavin', 'Niacin',
       'Pantothenic acid', 'Vitamin B-6', 'Folate, total', 'Vitamin B-12',
       'Vitamin K (phylloquinone)', 'Cholesterol', 'Fatty acids, total trans',
       'Fatty acids, total saturated']:
    all_data_large[j] = dict([(foodlarge[i], dietlarge[j][i]) for i in range(len(foodlarge))])

In [7]:
all_data_large.keys()

dict_keys(['Protein', 'Carbohydrate, by difference', 'Energy1', 'Water', 'Energy2', 'Calcium, Ca', 'Iron, Fe', 'Magnesium, Mg', 'Phosphorus, P', 'Potassium, K', 'Sodium, Na', 'Zinc, Zn', 'Copper, Cu', 'Manganese, Mn', 'Selenium, Se', 'Vitamin A, RAE', 'Vitamin E (alpha-tocopherol)', 'Vitamin D', 'Vitamin C, total ascorbic acid', 'Thiamin', 'Riboflavin', 'Niacin', 'Pantothenic acid', 'Vitamin B-6', 'Folate, total', 'Vitamin B-12', 'Vitamin K (phylloquinone)', 'Cholesterol', 'Fatty acids, total trans', 'Fatty acids, total saturated'])

In [8]:
# create the problem
prob2 = LpProblem('Minimizing_Cholesterol', LpMinimize)

In [9]:
# create variables to be optimized, variables must be 0 or bigger
variableslarge = LpVariable.dicts('type', foodlarge, 0)

# add objective function
prob2 += lpSum([variableslarge[i] \
                * all_data_large['Cholesterol'][i] for i in foodlarge]), 'total_cholesterol'

In [10]:
# add constraints, minimal and maximal
for k in ['Protein', 'Carbohydrate, by difference', 'Energy1', 'Water', \
          'Energy2', 'Calcium, Ca', 'Iron, Fe', 'Magnesium, Mg', \
          'Phosphorus, P', 'Potassium, K', 'Sodium, Na', \
          'Zinc, Zn', 'Copper, Cu', 'Manganese, Mn', 'Selenium, Se', \
          'Vitamin A, RAE', 'Vitamin E (alpha-tocopherol)', 'Vitamin D', \
          'Vitamin C, total ascorbic acid', 'Thiamin', 'Riboflavin', \
          'Niacin', 'Pantothenic acid', 'Vitamin B-6', 'Folate, total', \
          'Vitamin B-12', 'Vitamin K (phylloquinone)']:
    prob2 += lpSum([variableslarge[i]*all_data_large[k][i] for i in foodlarge]) >= dailyintakelarge[k].tolist()[0], 'min_'+k
    prob2 += lpSum([variableslarge[i]*all_data_large[k][i] for i in foodlarge]) <= dailyintakelarge[k].tolist()[2], 'max_'+k

In [11]:
prob2.solve()
print("Status:", LpStatus[prob2.status])

Status: Optimal


In [12]:
# Final result for the optimization to achieve minimal Cholesterol

print('Food selection to minimize Cholesterol:\n')
for v in prob2.variables():
    if v.varValue > 0:
        print(v.name, "=", v.varValue, 'servings')
print("\nTotal Cholesterol from this diet:", round(value(prob2.objective), 2))

Food selection to minimize Cholesterol:

type_Beans,_adzuki,_mature_seeds,_raw = 0.059863415 servings
type_Broccoli_raab,_raw = 0.069514608 servings
type_Cocoa_mix,_no_sugar_added,_powder = 0.42866218 servings
type_Egg,_white,_dried,_flakes,_glucose_reduced = 0.14694398 servings
type_Infant_formula,_MEAD_JOHNSON,_ENFAMIL,_NUTRAMIGEN,_with_iron,_p = 0.73805891 servings
type_Infant_formula,_NESTLE,_GOOD_START_ESSENTIALS__SOY,__with_iron, = 0.4258564 servings
type_Infant_formula,_ROSS,_ISOMIL,_with_iron,_powder,_not_reconstitu = 0.050114149 servings
type_Margarine_like_spread,_approximately_60%_fat,_tub,_soybean_(hyd = 0.15033656 servings
type_Mung_beans,_mature_seeds,_raw = 0.25918767 servings
type_Nuts,_mixed_nuts,_dry_roasted,_with_peanuts,_with_salt_added = 0.18052856 servings
type_Oil,_vegetable,_sunflower,_linoleic,_(hydrogenated) = 1.184482 servings
type_Seeds,_sunflower_seed_kernels,_dry_roasted,_with_salt_added = 0.10375187 servings
type_Snacks,_potato_chips,_fat_free,_made_with_

Again, most constraints need to be included to make it delicious. 